De acuerdo a la cantidad de puntos disponible que tiene cada socio, realizamos clusters de socios y extraemos esas bases a archivos CSV para que puedan ser subidos a Emblue y se realicen envíos de mail

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.axes as ax
import pyodbc
import math
import os
pd.set_option('display.max_columns', None)

In [2]:
DB = {'servername': '',
      'database': '',
      'usuario' :'',
      'contraseña':'',
      'Encrypt':'yes',
      'TrustServerCertificate':'no',
      'Connection Timeout':'30'}

# create the connection
conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] +\
                      ';UID=' + DB['usuario'] + ';PWD='+ DB['contraseña'] + ';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30')

# query db
sql = """

SELECT cpl.CustomerId, sum(cpl.Points) 'Puntos', c.FirstName, c.Email
FROM dbo.CustomerPointsLog cpl
INNER JOIN dbo.Customer c ON c.CustomerId = cpl.CustomerId
GROUP BY cpl.CustomerId, c.FirstName, c.Email
HAVING sum(cpl.Points) > 1799
"""
df = pd.read_sql(sql, conn)
df.head()

,CustomerId,Puntos,FirstName,Email
0,273AFE21-79E8-CC3A-AAF6-08D0C0CE8984,5440,Lujan Evangelina,lujantassone@hotmail.com
1,D1C0D365-6970-C40B-A6E6-08D0BD70A3EB,13720,Jose Antonio,joseamin12@hotmail.com
2,10C29CB8-C4F5-C071-3842-08D0BF4D95AE,5220,Luciana Mariela,acolombero@huincacoop.com.ar
3,98CAB44B-121A-C48F-E2F3-08D0BF43298F,2260,Melina,mely-2012-22@hotmail.com
4,6ACC86A5-45BC-C851-BE13-08D0BF45D4BD,6050,Carolina Marina,carolmon7@hotmail.com


In [3]:
#Como ya me pasan los datos que necesito para df, importo directamente.

#df = pd.read_csv(r'C:/Users/pnovero/Documents/Extractos DW/puntos al 07-12.csv')
#df.drop(index=df.index[0], axis=0, inplace=True)
#df["Puntos"] = df.Puntos.astype('int')

#df.head()

In [4]:
df.shape

(2617797, 4)

In [5]:
df.Puntos.fillna(0,inplace=True)
df.isna().sum().sort_values(ascending=False)

Email         3
FirstName     1
CustomerId    0
Puntos        0
dtype: int64

In [6]:
df.dropna(how="any",inplace=True)

In [7]:
pd.DataFrame(df[df.Puntos < 0])

,CustomerId,Puntos,FirstName,Email


In [8]:
df.drop(columns=['CustomerId'],inplace=True)

In [9]:
clusters = [-np.inf,-1,1799,4599,5999,6499,7999,np.inf]
nombres_clusters =["negativos","0 a 1799","1800 a 4599","4600 a 5999","6000 a 6499","6500 a 7999","mas de 8000"]
df['segmento_puntos'] = pd.cut(df.Puntos,bins= clusters, labels = nombres_clusters)

In [10]:
for grupo in ["1800 a 4599","4600 a 5999","6000 a 6499","6500 a 7999","mas de 8000"]:
    max_MB=50
    df_grupo = df[df["segmento_puntos"] == grupo]
    df_grupo.drop('segmento_puntos',axis=1,inplace=True)
    path = r'C:/Users/pnovero/Documents/Extractos DW/' + grupo + '.csv'
    df_grupo.to_csv(path,index=False)
    tamaño = os.path.getsize(path)/1000000
    print(tamaño)
    if tamaño > max_MB:
        os.remove(path)
        cantidad_archivos= math.ceil(tamaño/max_MB)
        archivos= np.array_split(df_grupo,cantidad_archivos)
        for idx,archivo in enumerate(archivos):
            path = r'C:/Users/pnovero/Documents/Extractos DW/' + grupo + '_' +str(idx) + '.csv'
            archivo.to_csv(path,index=False)
            print(path)

C:\Users\pnovero\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


56.061549
C:/Users/pnovero/Documents/Extractos DW/1800 a 4599_0.csv
C:/Users/pnovero/Documents/Extractos DW/1800 a 4599_1.csv
17.262759
4.636647
10.248487
19.275507
